In [1]:
import os
import sys
import time
import logging
import datetime
import argparse

home_dir = os.path.abspath(os.getcwd()+"/../")
sys.path.append(home_dir)
home_dir = os.path.abspath(os.getcwd())
sys.path.append(home_dir)
print(home_dir)

from pprint import pprint
from mpi4py import MPI
import numpy as np

import torch
from detectron2.data import MetadataCatalog
from detectron2.utils.logger import log_every_n_seconds

from utils.arguments import load_opt_command
from utils.distributed import init_distributed, is_main_process, apply_distributed, synchronize
from utils.misc import hook_metadata, hook_switcher, hook_opt
from datasets import build_evaluator, build_eval_dataloader
from xdecoder import build_model
from xdecoder.BaseModel import BaseModel
from xdecoder.utils import get_class_names

# logger = logging.getLogger(__name__)
# logging.basicConfig(level = logging.INFO)

from utils.arguments import load_opt_from_config_files, load_config_dict_to_opt

/home/djjin/Mygit/X-Decoder/notebooks


Invalid MIT-MAGIC-COOKIE-1 key/home/djjin/anaconda3/envs/conda_visual_HPE/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


datasets/ADE20K_2021_17_01/images_detectron2/training
datasets/ADE20K_2021_17_01/images_detectron2/validation


In [2]:
parser = argparse.ArgumentParser(description='Pretrain or fine-tune models for NLP tasks.')
parser.add_argument('--command', default="evaluate", help='Command: train/evaluate/train-and-evaluate')
parser.add_argument('--conf_files', nargs='+', help='Path(s) to the config file(s).')
parser.add_argument('--user_dir', help='Path to the user defined module for tasks (models, criteria), optimizers, and lr schedulers.')
parser.add_argument('--config_overrides', nargs='*', help='Override parameters on config with a json style string, e.g. {"<PARAM_NAME_1>": <PARAM_VALUE_1>, "<PARAM_GROUP_2>.<PARAM_SUBGROUP_2>.<PARAM_2>": <PARAM_VALUE_2>}. A key with "." updates the object in the corresponding nested dict. Remember to escape " in command line.')
parser.add_argument('--overrides', help='arguments that used to override the config file in cmdline', nargs=argparse.REMAINDER)

cmdline_args = parser.parse_args('')
cmdline_args.conf_files = [os.path.join(home_dir, "../configs/xdecoder/svlp_focalt_lang.yaml")]
cmdline_args.overrides = ['WEIGHT', '../checkpoints/xdecoder_focalt_best_openseg.pt'] 
cmdline_args.overrides

['WEIGHT', '../checkpoints/xdecoder_focalt_best_openseg.pt']

In [3]:

opt = load_opt_from_config_files(cmdline_args.conf_files)

keys = [cmdline_args.overrides[idx*2] for idx in range(len(cmdline_args.overrides)//2)]
vals = [cmdline_args.overrides[idx*2+1] for idx in range(len(cmdline_args.overrides)//2)]
vals = [val.replace('false', '').replace('False','') if len(val.replace(' ', '')) == 5 else val for val in vals]
types = []
for key in keys:
    key = key.split('.')
    ele = opt.copy()
    while len(key) > 0:
        ele = ele[key.pop(0)]
    types.append(type(ele))

config_dict = {x:z(y) for x,y,z in zip(keys, vals, types)}
config_dict

load_config_dict_to_opt(opt, config_dict)
for key, val in cmdline_args.__dict__.items():
    if val is not None:
        opt[key] = val

opt = init_distributed(opt)

In [4]:
pprint(opt)

{'ADE20K': {'DATASET': {'DATASET': 'ade'},
            'INPUT': {'COLOR_AUG_SSD': True,
                      'CROP': {'ENABLED': True,
                               'SINGLE_CATEGORY_MAX_AREA': 1.0,
                               'SIZE': '(640, 640)',
                               'TYPE': 'absolute'},
                      'DATASET_MAPPER_NAME': 'mask_former_panoptic',
                      'FORMAT': 'RGB',
                      'MASK_FORMAT': 'polygon',
                      'MAX_SIZE_TEST': 200,
                      'MAX_SIZE_TRAIN': 200,
                      'MIN_SIZE_TEST': 640,
                      'MIN_SIZE_TRAIN': 640,
                      'MIN_SIZE_TRAIN_SAMPLING': 'choice',
                      'SIZE_DIVISIBILITY': 640},
            'TEST': {'BATCH_SIZE_TOTAL': 4}},
 'BDD': {'DATALOADER': {'ASPECT_RATIO_GROUPING': False,
                        'FILTER_EMPTY_ANNOTATIONS': False,
                        'LOAD_PROPOSALS': False,
                        'NUM_WORKERS': 0,
 

In [5]:
model = BaseModel(opt, build_model(opt)).from_pretrained(opt['WEIGHT']).eval().cuda()

*UNLOADED* sem_seg_head.predictor.pos_embed_caping.weight, Model Shape: torch.Size([77, 512])
$UNUSED$ criterion.empty_weight, Ckpt Shape: torch.Size([134])
$UNUSED$ sem_seg_head.predictor.query_feat_caping.weight, Ckpt Shape: torch.Size([77, 512])


In [6]:
dataloaders = build_eval_dataloader(opt)
dataset_names = opt['DATASETS']['TEST']

In [7]:
for dataloader, dataset_name in zip(dataloaders, dataset_names):
    # build evaluator
    evaluator = build_evaluator(opt, dataset_name, opt['SAVE_DIR'])
    evaluator.reset()
    with torch.no_grad():
        for idx, batch in enumerate(dataloader):
            print(len(batch))
            # forward
            # with torch.autocast(device_type='cuda', dtype=torch.float16):
            #     outputs = model(batch, mode=eval_type)
            # print(outputs)

4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4


KeyboardInterrupt: 